In [ ]:
!pip install openai -q


In [ ]:
import pandas as pd
import numpy as np
import re

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

pd.set_option('max_colwidth', 200)
INPUT_PATH  = 'datasets/'
OUTPUT_PATH = 'outputs/'
SEED = 42

from openai import OpenAI
client=OpenAI(
    api_key ="INSERT API KEY HERE"
)

In [ ]:
# Insert appropriate dataset reading code from dataset_readers.py

def read_dataset():
    pass

prompt = '''INSERT PROMPT HERE'''

In [ ]:
train_df, test_df, label_names, dataset_name, text_col = read_dataset()

display(train_df.head())
display(test_df.head())
display(label_names)

In [ ]:
def predict_with_chatgpt(sentences: pd.Series, prompt: str) -> list[str]:
    
    def chat_with_chatgpt(prompt, model="gpt-3.5-turbo"):
        chat_completion=client.chat.completions.create(
            messages=[{
                "role":"user",
                "content":prompt,
            }],
            model="gpt-3.5-turbo"
        )
        message = chat_completion.choices[0].message.content
        return message
    
    # Get chatbot response via API
    responses = []
    for index, sentence in enumerate(sentences):
        gpt_prompt = prompt.format(sentence.strip().lower())
        chatbot_response = chat_with_chatgpt(gpt_prompt)
        responses.append(chatbot_response.strip())
        
        if index % 100 == 0:
            print(index)
    
    return responses


def parse_gpt_output(raw_responses: list[str], start_tag='<tr>', end_tag='</tr>'):

    parsed_responses = []
    for item in raw_responses:
        if start_tag not in item:
            output = ''
        else:
            parts  = item.split(start_tag)
            output = parts[1].split(end_tag)[0].strip()
        
        parsed_responses.append( output.lower() )
    
    return pd.Series(parsed_responses)

In [ ]:
# Translate train set
train_responses = predict_with_chatgpt(train_df.sentence, prompt)
train_responses_parsed = parse_gpt_output(train_responses)

train_df['translated'] = train_responses_parsed

train_df.to_csv(f'{OUTPUT_PATH}/{dataset_name}_train_df.csv', index=False)

with open(f'{dataset_name}_train_raw_responses.txt', 'w') as f:
    for line in train_responses:
        f.write(f"{line}\n")

In [ ]:
# Translate test set

test_responses = predict_with_chatgpt(test_df.sentence, prompt)
test_responses_parsed = parse_gpt_output(test_responses)

test_df['translated'] = test_responses_parsed

test_df.to_csv(f'{OUTPUT_PATH}/{dataset_name}_test_df.csv', index=False)

with open(f'{dataset_name}_test_raw_responses.txt', 'w') as f:
    for line in test_responses:
        f.write(f"{line}\n")